<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('max_columns', 150)

In [37]:
folder = './data/'
application_train = pd.read_csv(os.path.join(folder, 'application_train.csv'))
#application_test = pd.read_csv(os.path.join(folder, 'application_test.csv'))
bureau = pd.read_csv(os.path.join(folder, 'bureau.csv'))
# bureau_balance = pd.read_csv(os.path.join(folder, 'bureau_balance.csv'))
# POS_CASH_balance = pd.read_csv(os.path.join(folder, 'POS_CASH_balance.csv'))
# credit_card_balance = pd.read_csv(os.path.join(folder, 'credit_card_balance.csv'))
previous_application = pd.read_csv(os.path.join(folder, 'previous_application.csv'))
# installments_payments = pd.read_csv(os.path.join(folder, 'installments_payments.csv'))
# sample_submission = pd.read_csv(os.path.join(folder, 'sample_submission.csv'))

Let's create some aggreagated features!

* total count of loans in bureau, also separately in each status (credit_active);
* whether any credit was in a different currency;
* min, max, average of DAYS_CREDIT;
* were there any days overdue and max value of it;
* how long ago did previous and first cledits end;
* max overdue amount and max overdue amount relative to credit amount;
* were there prolongued credits;
* min, max, average of AMT_CREDIT_SUM;
* max AMT_CREDIT_SUM_DEBT;
* max AMT_CREDIT_SUM_LIMIT;
* count of credits of every type;
* days since DAYS_CREDIT_UPDATE;

In [20]:
bureau_agg = bureau[['SK_ID_CURR']].copy()
bureau_agg['loans_count'] = bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')
# There was an error, thanks to Fenglv for noticing
# bureau_agg[['Active_count', 'Bad_debt_count', 'Closed_count', 'Sold_count']] = pd.crosstab(bureau['SK_ID_CURR'], bureau['CREDIT_ACTIVE']).reset_index(drop=True)
bureau_agg = bureau_agg.join(pd.crosstab(bureau['SK_ID_CURR'], bureau['CREDIT_ACTIVE']), on='SK_ID_CURR')

bureau_agg['DAYS_CREDIT_mean'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('mean') * -1
bureau_agg['DAYS_CREDIT_max'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('max') * -1
bureau_agg['DAYS_CREDIT_min'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('min') * -1
bureau_agg['CREDIT_DAY_OVERDUE_max'] = bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform('max')
bureau_agg['DAYS_CREDIT_ENDDATE_min'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform('min')
bureau_agg['DAYS_CREDIT_ENDDATE_max'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform('max')
bureau_agg['AMT_CREDIT_MAX_OVERDUE_max'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform('max')
bureau_agg['CNT_CREDIT_PROLONG_max'] = bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform('max')
bureau_agg['AMT_CREDIT_SUM_mean'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('mean')
bureau_agg['AMT_CREDIT_SUM_max'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('max')
bureau_agg['AMT_CREDIT_SUM_min'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('min')
bureau_agg['AMT_CREDIT_SUM_DEBT_max'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform('max')
bureau_agg['AMT_CREDIT_SUM_LIMIT_max'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform('max')
bureau_agg['AMT_CREDIT_SUM_OVERDUE_max'] = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform('max')
bureau_agg['DAYS_CREDIT_UPDATE_max'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform('max') * -1

bureau_agg = bureau_agg.join(pd.crosstab(bureau['SK_ID_CURR'], bureau['CREDIT_TYPE']).reset_index(), on='SK_ID_CURR', rsuffix='y')
bureau_agg.drop('SK_ID_CURRy', axis=1, inplace=True)

bureau_agg.fillna(0, inplace=True)

In [21]:
bureau_active = bureau[bureau.CREDIT_ACTIVE=='Active']
bureau_active_agg = bureau_active[['SK_ID_CURR']].copy()
bureau_active_agg['loans_count'] = bureau_active.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

bureau_active_agg.loc[:,'DAYS_CREDIT_mean'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('mean') * -1
bureau_active_agg['DAYS_CREDIT_max'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('max') * -1
bureau_active_agg['DAYS_CREDIT_min'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform('min') * -1
bureau_active_agg['CREDIT_DAY_OVERDUE_max'] = bureau_active.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform('max')
bureau_active_agg['DAYS_CREDIT_ENDDATE_min'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform('min')
bureau_active_agg['DAYS_CREDIT_ENDDATE_max'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform('max')
bureau_active_agg['AMT_CREDIT_MAX_OVERDUE_max'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform('max')
bureau_active_agg['CNT_CREDIT_PROLONG_max'] = bureau_active.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform('max')
bureau_active_agg['AMT_CREDIT_SUM_mean'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('mean')
bureau_active_agg['AMT_CREDIT_SUM_max'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('max')
bureau_active_agg['AMT_CREDIT_SUM_min'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('min')
bureau_active_agg['AMT_CREDIT_SUM_DEBT_max'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform('max')
bureau_active_agg['AMT_CREDIT_SUM_LIMIT_max'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform('max')
bureau_active_agg['AMT_CREDIT_SUM_OVERDUE_max'] = bureau_active.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform('max')
bureau_active_agg['DAYS_CREDIT_UPDATE_max'] = bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform('max') * -1

bureau_active_agg = bureau_active_agg.join(pd.crosstab(bureau_active['SK_ID_CURR'], bureau_active['CREDIT_TYPE']).reset_index(), on='SK_ID_CURR', rsuffix='y')
bureau_active_agg.drop('SK_ID_CURRy', axis=1, inplace=True)

bureau_active_agg.fillna(0, inplace=True)


In [25]:
DAYS_CREDIT_mins = bureau.groupby('SK_ID_CURR').DAYS_CREDIT.transform(min)
bureau_last = bureau[bureau.DAYS_CREDIT == DAYS_CREDIT_mins].copy()

bureau_last.fillna(0, inplace=True)

In [30]:
bureau_last[['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_MAX_OVERDUE',
            'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE']]

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
11,Closed,currency 1,-2456,0,NaN,0,675000.000,0.00,0.00,0.0,Consumer credit
13,Active,currency 1,-96,0,0.000,0,89910.000,76905.00,0.00,0.0,Consumer credit
15,Closed,currency 1,-2911,0,NaN,0,48555.000,NaN,NaN,0.0,Consumer credit
24,Closed,currency 1,-2744,0,310.500,0,18157.500,NaN,NaN,0.0,Consumer credit
28,Closed,currency 1,-2534,0,NaN,0,38830.500,0.00,0.00,0.0,Consumer credit
32,Closed,currency 1,-559,0,0.000,0,110250.000,0.00,0.00,0.0,Credit card
33,Closed,currency 1,-2413,0,NaN,0,180000.000,0.00,0.00,0.0,Consumer credit
52,Closed,currency 1,-2898,0,NaN,0,11340.000,NaN,NaN,0.0,Consumer credit
54,Closed,currency 1,-1304,0,NaN,0,225000.000,0.00,0.00,0.0,Consumer credit
60,Closed,currency 1,-2785,0,NaN,0,225000.000,0.00,0.00,0.0,Consumer credit


In [40]:
prev_app_agg = previous_application[['SK_ID_CURR']].copy()

prev_app_agg['applications_count'] =  previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')
prev_app_agg = prev_app_agg.join(pd.crosstab(previous_application['SK_ID_CURR'], previous_application['NAME_CONTRACT_TYPE']), on='SK_ID_CURR')
prev_app_agg['AMT_ANNUITY_mean'] = previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform('mean') * -1
prev_app_agg['AMT_ANNUITY_max'] = previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform('max') * -1
prev_app_agg['AMT_ANNUITY_min'] = previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform('min') * -1
prev_app_agg['AMT_APPLICATION_max'] = previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform('max')
prev_app_agg['AMT_APPLICATION_min'] = previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform('min')
prev_app_agg['AMT_APPLICATION_mean'] = previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform('mean')
prev_app_agg['AMT_CREDIT_max'] = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform('max')
prev_app_agg['AMT_CREDIT_min'] = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform('min')
prev_app_agg['AMT_CREDIT_mean'] = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform('mean')
prev_app_agg['AMT_GOODS_PRICE_max'] = previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform('max')
prev_app_agg['AMT_GOODS_PRICE_min'] = previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform('min')
prev_app_agg['AMT_GOODS_PRICE_mean'] = previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform('mean')
prev_app_agg['DAYS_LAST_DUE_1ST_VERSION_max'] = previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform('max') * -1
prev_app_agg['DAYS_LAST_DUE_1ST_VERSION_min'] = previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform('min') * -1
prev_app_agg['DAYS_LAST_DUE_max'] = previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform('max') * -1
prev_app_agg['DAYS_LAST_DUE_min'] = previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform('min') * -1

prev_app_agg[['Approved_count', 'Canceled_count', 'Refused_count', 'Unused_offer_count']] = pd.crosstab(previous_application['SK_ID_CURR'], previous_application['NAME_CONTRACT_STATUS']).reset_index(drop=True)
prev_app_agg[['CLIENT', 'HC', 'LIMIT', 'SCO', 'SCOFR', 'SYSTEM', 'VERIF', 'XAP', 'XNA']] = pd.crosstab(previous_application['SK_ID_CURR'], previous_application['CODE_REJECT_REASON']).reset_index(drop=True)
prev_app_agg.fillna(0, inplace=True)



In [41]:
prev_app_agg

,SK_ID_CURR,applications_count,Cash loans,Consumer loans,Revolving loans,XNA,AMT_ANNUITY_mean,AMT_ANNUITY_max,AMT_ANNUITY_min,AMT_APPLICATION_max,AMT_APPLICATION_min,AMT_APPLICATION_mean,AMT_CREDIT_max,AMT_CREDIT_min,AMT_CREDIT_mean,AMT_GOODS_PRICE_max,AMT_GOODS_PRICE_min,AMT_GOODS_PRICE_mean,DAYS_LAST_DUE_1ST_VERSION_max,DAYS_LAST_DUE_1ST_VERSION_min,DAYS_LAST_DUE_max,DAYS_LAST_DUE_min,Approved_count,Canceled_count,Refused_count,Unused_offer_count,CLIENT,HC,LIMIT,SCO,SCOFR,SYSTEM,VERIF,XAP
0,271877,3,0,3,0,0.0,-27468.825000,-68258.655,-1730.430,1800000.0,17145.0,6.418485e+05,1754721.0,17145.0,6.305715e+05,1800000.0,17145.00,6.418485e+05,-300.0,182.0,42.0,392.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,108129,6,2,3,1,0.0,-13039.485000,-25188.615,-4830.930,607500.0,0.0,2.015258e+05,679671.0,23688.0,2.575222e+05,607500.0,33052.50,2.418309e+05,-365243.0,846.0,-365243.0,846.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,122040,4,2,1,1,0.0,-9623.115000,-15060.735,-6300.000,112500.0,0.0,6.669000e+04,136444.5,0.0,7.269638e+04,112500.0,58545.00,8.892000e+04,-365243.0,2215.0,-365243.0,2425.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,176158,23,22,1,0,0.0,-28022.556429,-56907.675,-10519.965,675000.0,0.0,2.597050e+05,709749.0,0.0,2.725785e+05,675000.0,105214.95,2.844388e+05,152.0,2389.0,182.0,2389.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,202054,25,18,6,1,0.0,-21187.627941,-42778.845,-3020.535,675000.0,0.0,1.790377e+05,744498.0,0.0,2.080616e+05,675000.0,14751.00,2.632908e+05,-365243.0,1484.0,-365243.0,1484.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5,199383,6,3,2,1,0.0,-15183.981000,-31527.765,-4991.535,315000.0,0.0,1.403618e+05,340573.5,0.0,1.492402e+05,315000.0,38250.00,1.684341e+05,-365243.0,1961.0,-365243.0,1961.0,5.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0
6,175704,4,1,2,1,0.0,-8826.090000,-12798.765,-6750.000,148248.0,0.0,8.656088e+04,135000.0,0.0,8.285400e+04,148248.0,62995.50,1.154145e+05,-365243.0,293.0,-365243.0,323.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
7,296299,17,14,2,1,0.0,-16069.250000,-25996.365,-3986.010,549000.0,0.0,1.221467e+05,681444.0,0.0,1.640899e+05,549000.0,19998.00,2.595617e+05,-365243.0,815.0,-365243.0,905.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
8,342292,14,6,7,1,0.0,-9816.185455,-21959.865,-2905.470,360000.0,0.0,1.002827e+05,393264.0,0.0,1.107225e+05,360000.0,17185.50,1.276326e+05,-690.0,2578.0,-365243.0,2578.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
9,334349,6,4,2,0,0.0,-12305.955000,-15157.935,-10475.640,346500.0,0.0,7.932000e+04,394524.0,0.0,8.631150e+04,346500.0,55260.00,1.586400e+05,-945.0,2568.0,-365243.0,2628.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


0     FIRST QUARTER OF THE YEAR
1    SECOND QUARTER OF THE YEAR
2    THIRD  QUARTER OF THE YEAR
3    FOURTH QUARTER OF THE YEAR
Name: Description, dtype: object
